# Data Source

Default of Credit Card Clients.

From: [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients)

# Problem Definition

Predict the payment status (default: yes or no) for the next month credit card bill statements of a customer.

The predictors used are some demographic data and the 6-month backtracking of the bill statements and payments of a customer.

# Data Collection

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Loading dataset

In [2]:
# load the dataset as pandas DataFrame
df = pd.read_excel('/home/er_bim/creditcard_payment_prediction/notebooks/data/default of credit card clients.xls', index_col=0)

# show the random data
df.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


## Preliminary dataset examination

The top row contains the column names that more understandable than the original notations, so I will convert them to the column names.

In [3]:
# set the top row data as column names
df.rename(columns=df.iloc[0], inplace = True)
df.drop(df.index[0], inplace = True)

In [4]:
# recheck the dataset
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [5]:
# check the dataset column names
df.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

The first column will be dropped and reset the dataset index.

In [6]:
# reset the dataset index
df.reset_index(inplace=True)
df.drop(columns=df.columns[0], axis=1, inplace=True)

Then we need to alter the inconsistent column name, `PAY_0` to `PAY_1`.

Also the target column will be renamed.

In [7]:
# define dictionary for column names alteration
cols = {'PAY_0': 'PAY_1', 'default payment next month': 'default_pred'}

# execute the column names alteration
df.rename(columns = cols, inplace = True)

# Data Definition


- LIMIT_BAL: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.
- SEX: Customer's gender (1 = male; 2 = female).
- EDUCATION: Customer's highest education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
- MARRIAGE: Customer's marital status (1 = married; 2 = single; 3 = others).
- AGE: Customer's age (years old).
- PAY_1: Customer's repayment status in one month before the current month.
- PAY_2: Customer's repayment status in two month before the current month.
- PAY_3: Customer's repayment status in three month before the current month.
- PAY_4: Customer's repayment status in four month before the current month.
- PAY_5 Customer's repayment status in five month before the current month.
- PAY_6: Customer's repayment status in six month before the current month.
- The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- BILL_AMT1: Amount of bill statement in one month before the current month (NT dollar).
- BILL_AMT2: Amount of bill statement in two month before the current month (NT dollar).
- BILL_AMT3: Amount of bill statement in three month before the current month (NT dollar).
- BILL_AMT4: Amount of bill statement in four month before the current month (NT dollar).
- BILL_AMT5: Amount of bill statement in five month before the current month (NT dollar).
- BILL_AMT6: Amount of bill statement in six month before the current month (NT dollar).
- PAY_AMT1: Amount paid in one month before the current month (NT dollar).
- PAY_AMT2: Amount paid in two month before the current month (NT dollar).
- PAY_AMT3: Amount paid in three month before the current month (NT dollar).
- PAY_AMT4: Amount paid in four month before the current month (NT dollar).
- PAY_AMT5: Amount paid in five month before the current month (NT dollar).
- PAY_AMT6: Amount paid in six month before the current month (NT dollar).
- default_pred_: The target column, **predicted payment status for the next month credit card bill statements of a customer ('Yes' = 1 and 'No' = 0)**.

# Data Evaluation

## Basic dataset information

In [8]:
print(f"The dataset contains of {df.shape[0]} columns and {df.shape[1]} rows")

The dataset contains of 30000 columns and 24 rows


In [9]:
print(f"The dataset contains {df.duplicated().sum()} duplicate value")

The dataset contains 35 duplicate value


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LIMIT_BAL     30000 non-null  object
 1   SEX           30000 non-null  object
 2   EDUCATION     30000 non-null  object
 3   MARRIAGE      30000 non-null  object
 4   AGE           30000 non-null  object
 5   PAY_1         30000 non-null  object
 6   PAY_2         30000 non-null  object
 7   PAY_3         30000 non-null  object
 8   PAY_4         30000 non-null  object
 9   PAY_5         30000 non-null  object
 10  PAY_6         30000 non-null  object
 11  BILL_AMT1     30000 non-null  object
 12  BILL_AMT2     30000 non-null  object
 13  BILL_AMT3     30000 non-null  object
 14  BILL_AMT4     30000 non-null  object
 15  BILL_AMT5     30000 non-null  object
 16  BILL_AMT6     30000 non-null  object
 17  PAY_AMT1      30000 non-null  object
 18  PAY_AMT2      30000 non-null  object
 19  PAY_

The data served in number characters but recognized as object data types, except the columns `sex`, `education`, and `marriage`, all data will be converted to numeric values.

No null value occurred in the dataset but there are some duplicated values.

We will correct them.

In [11]:
# # load the dataset as pandas DataFrame
# df = pd.read_excel('/home/er_bim/creditcard_payment_prediction/notebooks/data/default of credit card clients.xls', index_col=0)

# # set the top row data as column names
# df.rename(columns=df.iloc[0], inplace = True)
# df.drop(df.index[0], inplace = True)

# # reset the dataset index
# df.reset_index(inplace=True)
# df.drop(columns=df.columns[0], axis=1, inplace=True)

# # define dictionary for column names alteration
# cols = {'PAY_0': 'PAY_1', 'default payment next month': 'default_pred'}

# # execute the column names alteration
# df.rename(columns = cols, inplace = True)

# # drop the duplicated data
# df.drop_duplicates(inplace=True, ignore_index=True)

# # alter to numeric data type 
# df = df.apply(pd.to_numeric)

# # define the categorical columns
# cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default_pred']
# df[cat_cols] = df[cat_cols].astype(str)

In [12]:
# drop the duplicated data
df.drop_duplicates(inplace=True, ignore_index=True)

In [13]:
# alter to numeric data type 
df = df.apply(pd.to_numeric)

# define the categorical columns
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default_pred']
df[cat_cols] = df[cat_cols].astype(str)

## Statistical summary

In [14]:
# statistical summary for categorical columns
df.describe(exclude=[np.number])

,SEX,EDUCATION,MARRIAGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default_pred
count,29965,29965,29965,29965,29965,29965,29965,29965,29965,29965
unique,2,7,4,11,11,11,11,10,10,2
top,2,2,2,0,0,0,0,0,0,0
freq,18091,14019,15945,14737,15730,15764,16455,16947,16286,23335


In [15]:
# statistical summary for numeric columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
LIMIT_BAL,29965.0,167442.005006,129760.135222,10000.0,50000.0,140000.0,240000.0,1000000.0
AGE,29965.0,35.487969,9.219459,21.0,28.0,34.0,41.0,79.0
BILL_AMT1,29965.0,51283.009778,73658.132403,-165580.0,3595.0,22438.0,67260.0,964511.0
BILL_AMT2,29965.0,49236.366294,71195.567392,-69777.0,3010.0,21295.0,64109.0,983931.0
BILL_AMT3,29965.0,47067.916069,69371.352323,-157264.0,2711.0,20135.0,60201.0,1664089.0
BILL_AMT4,29965.0,43313.329885,64353.514373,-170000.0,2360.0,19081.0,54601.0,891586.0
BILL_AMT5,29965.0,40358.334390,60817.130623,-81334.0,1787.0,18130.0,50247.0,927171.0
BILL_AMT6,29965.0,38917.012281,59574.147742,-339603.0,1262.0,17124.0,49252.0,961664.0
PAY_AMT1,29965.0,5670.099316,16571.849467,0.0,1000.0,2102.0,5008.0,873552.0
PAY_AMT2,29965.0,5927.983180,23053.456645,0.0,850.0,2010.0,5000.0,1684259.0


Notes:
- The alterations to categorical columns aim to evaluate the data distribution and further in the EDA process; when later the data is already cleaned up and ready to be trained, they will be reconverted to numeric again.
- The mode value in the historical repayment status columns is '0' (zero), which is not defined in the data definition.
- There are negative values in the historical bill amount columns; my initial guess is caused by the customers paying more than they billed; however, we will investigate further.
- The data is **imbalance**; the customers predicted to not have a default payment are a very dominant population, 23335 out of 29965 customers.  

## Categorical features evaluation

In [16]:
# separate the numeric and categorical features
num_cols = df._get_numeric_data().columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

In [17]:
# Checking unique values in each categorical feature
for uv in cat_cols:
    uniques = df[uv].unique()
    print(f"The unique values in columns {uv} are: \n{uniques}\n\n")

The unique values in columns SEX are: 
['2' '1']


The unique values in columns EDUCATION are: 
['2' '1' '3' '5' '4' '6' '0']


The unique values in columns MARRIAGE are: 
['1' '2' '3' '0']


The unique values in columns PAY_1 are: 
['2' '-1' '0' '-2' '1' '3' '4' '8' '7' '5' '6']


The unique values in columns PAY_2 are: 
['2' '0' '-1' '-2' '3' '5' '7' '4' '1' '6' '8']


The unique values in columns PAY_3 are: 
['-1' '0' '2' '-2' '3' '4' '6' '7' '1' '5' '8']


The unique values in columns PAY_4 are: 
['-1' '0' '-2' '2' '3' '4' '5' '7' '6' '1' '8']


The unique values in columns PAY_5 are: 
['-2' '0' '-1' '2' '3' '5' '4' '7' '8' '6']


The unique values in columns PAY_6 are: 
['-2' '2' '0' '-1' '3' '6' '4' '7' '8' '5']


The unique values in columns default_pred are: 
['1' '0']




Notes:
- No problem with the column `sex`, the unique values are the same as stated in data definition.
- In the column `education`, the values 0, 5, and 6 will be categorized as value 4, which means 'others'.
- In the column `marriage`, the values 0 will be categorized as value 3, which means 'others'.
- In the columns refer to historical repayment status, the values -2, and 0 is confusing, we need to investigate it further.
- No problem with the column `default_pred`, the unique values are the same as stated in data definition.

## Historical bill and payment features evaluation

Now let's check the columns related to the customers' historical transactions. The 18 columns consist of: repayment status, bill statements, and bill payment amount of the last six months.

In [18]:
df[(df['PAY_1'] == '-1') & 
   (df['PAY_2'] == '-1') & 
   (df['PAY_3'] == '-1') &
   (df['PAY_4'] == '-1') & 
   (df['PAY_5'] == '-1') & 
   (df['PAY_6'] == '-1')].head(15).T

,21,28,30,48,70,74,77,106,107,116,118,141,171,206,207
LIMIT_BAL,120000,50000,230000,380000,80000,340000,300000,80000,350000,240000,400000,180000,300000,100000,210000
SEX,2,2,2,1,1,1,2,2,1,1,1,1,2,1,2
EDUCATION,2,3,1,2,1,1,1,2,1,1,2,2,1,1,2
MARRIAGE,1,1,2,2,2,2,1,1,2,2,1,2,2,1,1
AGE,39,47,27,32,31,32,45,36,52,28,34,28,30,47,41
PAY_1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
PAY_2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
PAY_3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
PAY_4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
PAY_5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


The table above shows us 10 customers with the best historical performance on paying the credit card transaction; they paid every bill duly for the last 6 months. So 90% of them is predicted to not have a default payment next month. Let's analyze the correlation between 'BILL_AMT' and 'PAY_AMT' columns.

The bill statements on M minus 6 (BILL_AMT6) column is paid on M minus 5 (PAY_AMT5), then the bill statements on M minus 5 (BILL_AMT5) column is paid on M minus 4 (PAY_AMT4), and so on until the bill statements on M minus 2 (BILL_AMT2) column is paid on M minus 1 (PAY_AMT1).

It means the value in column PAY_AMT6 refers to the customer's payment for the bill statement on M minus 7, which the data is not available in this dataset. So the value zero in column PAY_AMT6 means the customer was not paying the bill at all or the customer just started using the credit card 6 months ago.

About the column BILL_AMT1, it refers to the last month's (M minus 1) bill statement, for which the dataset does not cover the information of its payment. So the column BILL_AMT1 value justifies only the transaction activity of the customer until last month.


In [19]:
# check the customers whom paid late for 2 months every bill statement for the last 6 months
df[(df['PAY_2'] == '2') & 
   (df['PAY_2'] == '2') &
   (df['PAY_3'] == '2') & 
   (df['PAY_4'] == '2') &
   (df['PAY_5'] == '2') & 
   (df['PAY_6'] == '2')].head(20).T

,62,71,90,187,195,292,329,399,404,410,435,456,483,501,513,539,605,614,648,668
LIMIT_BAL,50000,320000,200000,200000,50000,30000,150000,120000,160000,60000,270000,130000,90000,90000,50000,110000,50000,190000,280000,100000
SEX,1,1,1,2,2,1,1,2,2,2,2,1,1,2,1,2,1,2,2,2
EDUCATION,1,2,1,3,1,2,1,2,1,1,2,3,3,2,2,2,1,2,2,3
MARRIAGE,2,2,1,2,2,1,1,2,2,2,1,2,1,1,2,1,2,1,1,1
AGE,29,29,53,47,24,43,40,25,29,23,32,39,48,35,23,40,34,37,44,35
PAY_1,2,2,2,2,1,3,2,2,2,1,1,1,1,1,2,1,1,1,2,1
PAY_2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
PAY_3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
PAY_4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
PAY_5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


The table above shows us 10 customers who consistently paid 2 months late for the last 6 months.

As inspected, they did not fully pay their transaction until last month; in other words, they currently have the outstanding credit.

To determine the data consistency, we try to compare with the costumers who paid more than 2 months late. 

In [20]:
# check the customers whom paid late for more than 2 months every bill statement for the last 6 months
df[(df['PAY_2'] > '2') & 
   (df['PAY_2'] > '2') &
   (df['PAY_3'] > '2') & 
   (df['PAY_4'] > '2') &
   (df['PAY_5'] > '2') & 
   (df['PAY_6'] > '2')].head(20).T

,158,649,749,831,981,1132,1894,2361,2915,4335,4386,4443,6749,8172,8837,8991,9073,9973,11547,12253
LIMIT_BAL,210000,20000,50000,20000,30000,270000,100000,20000,20000,330000,90000,100000,20000,70000,40000,80000,50000,30000,150000,90000
SEX,1,2,2,2,1,1,1,1,1,2,1,1,2,2,2,1,1,1,1,2
EDUCATION,3,2,2,3,2,1,2,2,3,2,2,2,2,2,3,2,2,1,1,2
MARRIAGE,1,2,2,1,2,1,2,2,2,1,1,2,1,1,1,2,2,1,1,2
AGE,45,46,25,24,29,44,30,24,26,39,35,36,44,32,24,31,26,38,42,23
PAY_1,2,8,8,8,8,3,1,8,8,8,8,8,8,3,1,8,1,8,8,2
PAY_2,3,7,7,7,7,3,3,7,7,7,7,7,7,3,8,7,5,7,7,4
PAY_3,4,6,6,6,6,4,7,6,6,6,6,6,6,4,7,6,4,6,6,4
PAY_4,4,5,5,5,5,4,6,5,5,5,5,5,5,4,6,5,8,5,5,3
PAY_5,5,4,4,4,4,4,7,4,4,4,4,4,4,5,5,4,7,4,4,4


From the table above, we can conclude that the customers that paid the bill lately tend to have a current outstanding unpaid credit, **a non-performing loan condition**.

I also get a unique finding from those two tables above: some of the customers that have a large amount of non-performing loan conditions are still predicted to not have the default payment; zero value in column `default_pred`. This kind of customer's characteristics are surely predicted to have the default payment.

Now let's check the columns refer to the customers' repayment status, which value is not defined in the data definition, 0 and -2.

In [21]:
# check the customers with the latest repayment status '0'
df[df['PAY_1'] == '0'].head(15).T

,2,3,5,6,7,8,10,14,16,17,20,24,25,27,29
LIMIT_BAL,90000,50000,50000,500000,100000,140000,200000,250000,20000,320000,130000,90000,50000,50000,50000
SEX,2,2,1,1,2,2,2,1,1,1,2,1,1,2,1
EDUCATION,2,2,1,1,2,3,3,1,1,1,3,1,3,3,1
MARRIAGE,2,1,2,2,2,1,2,2,2,1,2,2,2,2,2
AGE,34,37,37,29,23,28,34,29,24,49,39,23,23,30,26
PAY_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PAY_2,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0
PAY_3,0,0,0,0,-1,2,2,0,2,0,0,0,0,0,0
PAY_4,0,0,0,0,0,0,0,0,2,-1,0,-1,0,0,0
PAY_5,0,0,0,0,0,0,0,0,2,-1,0,0,0,0,0


In [22]:
# check the customers with the latest repayment status '-2'
df[df['PAY_1'] == '-2'].head(15).T

,9,23,33,34,45,52,55,65,92,100,101,143,166,176,196
LIMIT_BAL,20000,450000,500000,500000,210000,310000,500000,200000,100000,140000,380000,170000,370000,80000,150000
SEX,1,2,2,1,1,2,2,1,2,1,2,1,1,1,2
EDUCATION,3,1,2,1,1,2,1,1,1,1,1,2,1,1,2
MARRIAGE,2,1,1,1,2,1,1,1,2,2,2,2,2,2,1
AGE,35,40,54,58,29,49,45,57,27,32,30,29,50,30,34
PAY_1,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2
PAY_2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-2
PAY_3,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-2,-2,0,-2
PAY_4,-2,-2,-2,-2,-2,-2,-2,-1,-2,-2,0,-2,-2,0,-2
PAY_5,-1,-2,-2,-2,-2,-2,-2,2,-2,-2,0,-2,-2,0,-2


The customers with repayment status 0 are more likely to have the same characteristics as the customers with repayment status 2, which the table already showed above; they still pay their bills but not in full amount, and they currently have the outstanding credit.

On the contrary, the customers with repayment status -2 are more likely to have the same characteristics as the customers with repayment status 2. They tend to pay their bills duly, but they are less actively using the credit card, as shown by the many zero values, either in the bill statement or in the pay amount columns.

### Conclusion

- **The most reliable (and related) value in the repayment status columns is -1**, which means that the customer paid the bills duly.
- The customers with repayment status 2 are tend pay their bills but not in full amount, even worse for customers with repayment status more than 2 that many of them are still actively using the credit card but not paid the bills at all. These customers currently have the outstanding credit, a **non-performing loan condition**.
- From the customers with a non-performing loan condition, we can briefly inspected that some them are still predicted to be a non-deafult payment. We later **will examine to modify the false target column** based on the general condition in the industry.

# Feature Engineering

The columns related to the historical repayment status (PAY_1 to PAY_6) are not explaining completetely about the relationship between the customers' historical bills and payments to the default prediction.

The works I will accomplish in this section are:
- Add new feature that shows the credit balance of the customer, calculated from the historical bills and payments.
- Evaluate some customers default prediction by specific characteristics, and change the default prediction if necessary.
- Evaluate the unique values in columns `EDUCATION` and `MARRIAGE`, and adjust them to the values defined in then data definition.

We need to reload the dataset that will be manipulated in this section.

In [23]:
# load the dataset as pandas DataFrame
df_mod = pd.read_excel('/home/er_bim/creditcard_payment_prediction/notebooks/data/default of credit card clients.xls', index_col=0)

# set the top row data as column names
df_mod.rename(columns=df_mod.iloc[0], inplace = True)
df_mod.drop(df_mod.index[0], inplace = True)

# reset the dataset index
df_mod.reset_index(inplace=True)
df_mod.drop(columns=df_mod.columns[0], axis=1, inplace=True)

# define dictionary for column names alteration
cols = {'PAY_0': 'PAY_1', 'default payment next month': 'default_pred'}

# execute the column names alteration
df_mod.rename(columns = cols, inplace = True)

# drop the duplicated data
df_mod.drop_duplicates(inplace=True, ignore_index=True)

# alter to numeric data type 
df_mod = df_mod.apply(pd.to_numeric)

# define the categorical columns
cat_cols = ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'default_pred']
df_mod[cat_cols] = df_mod[cat_cols].astype(str)

## Add `balance` column

The idea is to find out each customer's current credit balance.

The column `PAY_AMT6` is excluded from this calculation because it is related to bill statements amount of 7 moths ago, which the data is not available in this dataset.

$$CURRENTBAL = {\displaystyle\sum_{i=1}^{6} BILLAMT_i} - {\displaystyle\sum_{i=1}^{5} PAYAMT_i}$$

In [24]:
# create a new column, current balance
df_mod['CURRENT_BAL'] = (df_mod.iloc[:, 11:17].sum(axis=1))-(df_mod.iloc[:, 17:22].sum(axis=1))

# reorder the target column
df_mod.insert(24, 'default_pred', df_mod.pop('default_pred'))

From this `CURRENT_BAL` column, we can compare to the `LIMIT_BAL` column. If the current balance (that is accumulated from a 6-month transaction) is more than the limit balance, then the customer meets the non-performing loan condition, a strong indicator for next month's default payment.

Let's check them.

In [25]:
# show the customers with non-performing loan
df_mod[df_mod['LIMIT_BAL'] < df_mod['CURRENT_BAL']].head(18).T

,2,3,4,5,6,13,14,15,16,17,22,25,27,29,31,32,36,40
LIMIT_BAL,90000,50000,50000,50000,500000,70000,250000,50000,20000,320000,70000,50000,50000,50000,50000,100000,280000,360000
SEX,2,2,1,1,1,1,1,2,1,1,2,1,2,1,1,1,1,1
EDUCATION,2,2,2,1,1,2,1,3,1,1,2,3,3,1,2,1,2,1
MARRIAGE,2,1,1,2,2,2,2,3,2,1,2,2,2,2,2,2,1,2
AGE,34,37,57,37,29,30,29,23,24,49,26,23,30,26,33,32,40,33
PAY_1,0,0,-1,0,0,1,0,1,0,0,2,0,0,0,2,0,0,0
PAY_2,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0
PAY_3,0,0,-1,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0
PAY_4,0,0,0,0,0,0,0,0,2,-1,2,0,0,0,0,0,0,0
PAY_5,0,0,0,0,0,0,0,0,2,-1,2,0,0,0,0,0,0,0


As we can see in the table above, there are many false predictions, the customers with non-performing loan predicted to nat have a deafult payment.

## Target column evaluation

Now we evaluate the target column correctness. There are some conditions that the prediction should be default but stated the opposite in the dataset.

- The non-performing loan condition: customers with the current balance higher than the limit.
- The non-paying customers: customers with active transactions but have zero payments in the last three months.
- The inactive customers: customers with no activity in the last 6 months, both bills and payments are zero.
- The duly-payers: customers with active transactions and paid duly every month usually have the current balance lower than the limit.

### The non-performing loan condition

As we defined in the previous sub-section, the non-performing loan condition is a strong indicator of the customer's default payment.

The customers that were predicted as non-defaults will be modified accordingly.

In [26]:
# define the non-performing loan table
outstanding = df_mod[df_mod['CURRENT_BAL'] > df_mod['LIMIT_BAL']]

print(f"There are {len(outstanding)} customers that have non-performing loan condition in the dataset")

There are 16350 customers that have non-performing loan condition in the dataset


In [27]:
# Calculate the count of each unique value in the target column
target_counts_outstanding = outstanding['default_pred'].value_counts().reset_index()

# Calculate the count distribution
total_counts = outstanding['default_pred'].value_counts().sum()
target_counts_outstanding['percentage'] = (target_counts_outstanding['count'] / total_counts) * 100

# Rename the columns
target_counts_outstanding.columns = ['default_pred', 'count', 'percentage']

# Display the table
print(f"The default_pred distribution of the customers with non-performing loan is:\n\n{target_counts_outstanding}")

The default_pred distribution of the customers with non-performing loan is:

  default_pred  count  percentage
0            0  12153   74.330275
1            1   4197   25.669725


In this condition, the true prediction should be default, thus the customers predicted as non-default will be altered accordingly.

In [28]:
# get the rows of this group
idx = outstanding.index

# replace the value in target column
df_mod.default_pred.loc[idx] = df_mod.default_pred.loc[idx].replace('0','1')

### The non-paying cutomers

Now we check the customers that are actively using the card but have been not paying their bills for at least 3 last months.

In [29]:
# define the non-paying customers table
non_pay = df_mod[(df_mod['PAY_AMT1'] == 0) & (df_mod['PAY_AMT2'] == 0) & (df_mod['PAY_AMT3'] == 0)
             & (df_mod['BILL_AMT1'] > 0) & (df_mod['BILL_AMT2'] > 0) & (df_mod['BILL_AMT3'] > 0) & (df_mod['BILL_AMT4'] > 0)]

print(f"There are {len(non_pay)} non-paying customers in the dataset")

There are 269 non-paying customers in the dataset


In [30]:
# Calculate the count of each unique value in the target column
target_counts_non_pay = non_pay['default_pred'].value_counts().reset_index()

# Calculate the count distribution
total_counts = non_pay['default_pred'].value_counts().sum()
target_counts_non_pay['percentage'] = (target_counts_non_pay['count'] / total_counts) * 100

# Rename the columns
target_counts_non_pay.columns = ['default_pred', 'count', 'percentage']

# Display the table
print(f"The default_pred distribution of the non-payers is:\n\n{target_counts_non_pay}")

The default_pred distribution of the non-payers is:

  default_pred  count  percentage
0            1    234   86.988848
1            0     35   13.011152


In this condition, the true prediction should be default, thus the customers predicted as non-default will be altered accordingly.

In [31]:
# get the rows of this group
idx = non_pay.index

# replace the value in target column
df_mod.default_pred.loc[idx] = df_mod.default_pred.loc[idx].replace('0','1')

### The inactive customers

Let's check the inactive customers, whose the bills and payments valued zero in the last 6 months.

In [32]:
# define the inactive customers
inactive = df_mod[(df_mod['PAY_AMT1'] == 0) & (df_mod['PAY_AMT2'] == 0) & (df_mod['PAY_AMT3'] == 0) & (df_mod['PAY_AMT4'] == 0) & (df_mod['PAY_AMT5'] == 0) & (df_mod['PAY_AMT6'] == 0)
            & (df_mod['BILL_AMT1'] == 0) & (df_mod['BILL_AMT2'] == 0) & (df_mod['BILL_AMT3'] == 0) & (df_mod['BILL_AMT4'] == 0) & (df_mod['BILL_AMT5'] == 0) & (df_mod['BILL_AMT6'] == 0)]
            
print(f"There are {len(inactive)} inactive customers in the dataset")

There are 765 inactive customers in the dataset


In [33]:
# Calculate the count of each unique value in the target column
target_counts_inactive = inactive['default_pred'].value_counts().reset_index()

# Calculate the count distribution
total_counts = inactive['default_pred'].value_counts().sum()
target_counts_inactive['percentage'] = (target_counts_inactive['count'] / total_counts) * 100

# Rename the columns
target_counts_inactive.columns = ['default_pred', 'count', 'percentage']

# Display the table
print(f"The default_pred distribution of the inactive customers is:\n\n{target_counts_inactive}")

The default_pred distribution of the inactive customers is:

  default_pred  count  percentage
0            0    471   61.568627
1            1    294   38.431373


In this condition, the true prediction should be non-default, thus the customers predicted as default will be altered accordingly.

In [34]:
# get the rows of this group
idx = inactive.index

# replace the value in target column
df_mod.default_pred.loc[idx] = df_mod.default_pred.loc[idx].replace('1','0')

### The duly-payers

Now we check the customers who paid the bills duly for the last 3 months, despite having any outstanding bills in the months before.

In [35]:
# check the customers whom paid duly every bill statement for the last 3 months
duly_payers = df_mod[(df_mod['PAY_1'] == '-1') & (df_mod['PAY_2'] == '-1') & (df_mod['PAY_3'] == '-1')]

print(f"There are {len(duly_payers)} duly-payers customers in the dataset")

There are 3691 duly-payers customers in the dataset


In [36]:
# Calculate the count of each unique value in the target column
target_counts_duly_payers = duly_payers['default_pred'].value_counts().reset_index()

# Calculate the count distribution
total_counts = duly_payers['default_pred'].value_counts().sum()
target_counts_duly_payers['percentage'] = (target_counts_duly_payers['count'] / total_counts) * 100

# Rename the columns
target_counts_duly_payers.columns = ['default_pred', 'count', 'percentage']

# Display the table
print(f"The default_pred distribution of the duly-payers is:\n\n{target_counts_duly_payers}")

The default_pred distribution of the duly-payers is:

  default_pred  count  percentage
0            0   3102   84.042265
1            1    589   15.957735


In [37]:
# get the rows of this group
idx = duly_payers.index

# replace the value in target column
df_mod.default_pred.loc[idx] = df_mod.default_pred.loc[idx].replace('1','0')

## Columns `EDUCATION` and `MARRIAGE` evaluation

The unique values defined in the data defintion for these columns are:
- EDUCATION: Customer's highest education (1 = graduate school; 2 = university; 3 = high school; 4 = others).
- MARRIAGE: Customer's marital status (1 = married; 2 = single; 3 = others).

Now let's their occurence in the dataset.

In [38]:
print(f"The unique values in column 'EDUCATION' are:\n{df_mod['EDUCATION'].unique()}\n")

print(f"The unique values in column 'MARRIAGE' are:\n{df_mod['EDUCATION'].unique()}")

The unique values in column 'EDUCATION' are:
['2' '1' '3' '5' '4' '6' '0']

The unique values in column 'MARRIAGE' are:
['2' '1' '3' '5' '4' '6' '0']


The values other than the ones defined will be altered to the 'others' category in each column.

In the `EDUCATION` column, the values: 0, 5, and 6 will be categorized as value 4 (others); while in the `MARRIAGE` column, the values: 0, 4, 5, and 6 will be categorized as value 3 (others).

In [39]:
# create the dictionary for value replacement
edu = ['0','5','6']
mar = ['0', '4', '5','6']

# execute the value replacements
df_mod.EDUCATION.replace(edu, '4', inplace=True)
df_mod.MARRIAGE.replace(mar, '3', inplace=True)

## Summary of feature engineering

We have done the feature engineering in this dataset, now let's check the results and compare them against the condition before.

### Numerical features

The only significant difference after the feature engineering is addition of current balance column.

The comparison is shown in the following tables:

In [40]:
# print the basic statistical summary BEFORE the feature engineering
df.describe().T

,count,mean,std,min,25%,50%,75%,max
LIMIT_BAL,29965.0,167442.005006,129760.135222,10000.0,50000.0,140000.0,240000.0,1000000.0
AGE,29965.0,35.487969,9.219459,21.0,28.0,34.0,41.0,79.0
BILL_AMT1,29965.0,51283.009778,73658.132403,-165580.0,3595.0,22438.0,67260.0,964511.0
BILL_AMT2,29965.0,49236.366294,71195.567392,-69777.0,3010.0,21295.0,64109.0,983931.0
BILL_AMT3,29965.0,47067.916069,69371.352323,-157264.0,2711.0,20135.0,60201.0,1664089.0
BILL_AMT4,29965.0,43313.329885,64353.514373,-170000.0,2360.0,19081.0,54601.0,891586.0
BILL_AMT5,29965.0,40358.334390,60817.130623,-81334.0,1787.0,18130.0,50247.0,927171.0
BILL_AMT6,29965.0,38917.012281,59574.147742,-339603.0,1262.0,17124.0,49252.0,961664.0
PAY_AMT1,29965.0,5670.099316,16571.849467,0.0,1000.0,2102.0,5008.0,873552.0
PAY_AMT2,29965.0,5927.983180,23053.456645,0.0,850.0,2010.0,5000.0,1684259.0


In [41]:
# print the basic statistical summary AFTER the feature engineering
df_mod.describe().T

,count,mean,std,min,25%,50%,75%,max
LIMIT_BAL,29965.0,167442.005006,129760.135222,10000.0,50000.0,140000.0,240000.0,1000000.0
AGE,29965.0,35.487969,9.219459,21.0,28.0,34.0,41.0,79.0
BILL_AMT1,29965.0,51283.009778,73658.132403,-165580.0,3595.0,22438.0,67260.0,964511.0
BILL_AMT2,29965.0,49236.366294,71195.567392,-69777.0,3010.0,21295.0,64109.0,983931.0
BILL_AMT3,29965.0,47067.916069,69371.352323,-157264.0,2711.0,20135.0,60201.0,1664089.0
BILL_AMT4,29965.0,43313.329885,64353.514373,-170000.0,2360.0,19081.0,54601.0,891586.0
BILL_AMT5,29965.0,40358.334390,60817.130623,-81334.0,1787.0,18130.0,50247.0,927171.0
BILL_AMT6,29965.0,38917.012281,59574.147742,-339603.0,1262.0,17124.0,49252.0,961664.0
PAY_AMT1,29965.0,5670.099316,16571.849467,0.0,1000.0,2102.0,5008.0,873552.0
PAY_AMT2,29965.0,5927.983180,23053.456645,0.0,850.0,2010.0,5000.0,1684259.0


### Categorical features

Now let's check the categorical features comparison after the modification in columns `EDUCATION`, `MARRIAGE` and `default_pred`.

In [42]:
# print the basic statistical summary BEFORE the feature engineering
df.describe(exclude=[np.number])

,SEX,EDUCATION,MARRIAGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default_pred
count,29965,29965,29965,29965,29965,29965,29965,29965,29965,29965
unique,2,7,4,11,11,11,11,10,10,2
top,2,2,2,0,0,0,0,0,0,0
freq,18091,14019,15945,14737,15730,15764,16455,16947,16286,23335


In [43]:
# print the basic statistical summary AFTER the feature engineering
df_mod.describe(exclude=[np.number])

,SEX,EDUCATION,MARRIAGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default_pred
count,29965,29965,29965,29965,29965,29965,29965,29965,29965,29965
unique,2,4,3,11,11,11,11,10,10,2
top,2,2,2,0,0,0,0,0,0,1
freq,18091,14019,15945,14737,15730,15764,16455,16947,16286,17935


### Save the modified dataset

In [44]:
# save the dataset
df_mod.to_csv('/home/er_bim/creditcard_payment_prediction/notebooks/data/modified_creditcard_data.csv', index=False)